# Bronze Layer

In [27]:
import sqlite3

# Sample data with a unique identifier (order_id)
data = [
    (1, 'C001', '2024-01-15', 250.50, 'completed'),
    (2, 'C002', '2024-02-20', 145.00, 'completed'),
    (3, 'C001', '2024-02-20', 300.00, 'pending'),
    (4, 'C003', '2024-03-05', 500.00, 'completed'),
    (5, 'C002', '2024-03-10', 200.00, 'cancelled'),
    (6, 'C001', '2024-03-15', 275.00, 'completed')
]

# Connect to the SQLite database
conn = sqlite3.connect('bronze_layer.db')
cursor = conn.cursor()

# Create the table with a primary key on `order_id`
cursor.execute('''
CREATE TABLE IF NOT EXISTS customer_order (
    order_id INTEGER PRIMARY KEY,
    customer_id TEXT,
    order_date TEXT,
    amount REAL,
    status TEXT
)
''')

for row in data:
    try:
        cursor.execute('INSERT INTO customer_order VALUES (?, ?, ?, ?, ?)', row)
    except sqlite3.IntegrityError:
        # Skip insertion if duplicate order_id is found
        print(f"Duplicate entry found for order_id {row[0]}; skipping insertion.")

# Commit the transaction and close the connection
conn.commit()
conn.close()



Duplicate entry found for order_id 1; skipping insertion.
Duplicate entry found for order_id 2; skipping insertion.
Duplicate entry found for order_id 3; skipping insertion.
Duplicate entry found for order_id 4; skipping insertion.
Duplicate entry found for order_id 5; skipping insertion.
Duplicate entry found for order_id 6; skipping insertion.


In [28]:
conn = sqlite3.connect('bronze_layer.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM customer_order;''')
# Fetch and print all the results
rows = cursor.fetchall()

# Check if there are results and display them
if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn.close()


(1, 'C001', '2024-01-15', 250.5, 'completed')
(2, 'C002', '2024-02-20', 145.0, 'completed')
(3, 'C001', '2024-02-20', 300.0, 'pending')
(4, 'C003', '2024-03-05', 500.0, 'completed')
(5, 'C002', '2024-03-10', 200.0, 'cancelled')
(6, 'C001', '2024-03-15', 275.0, 'completed')


# Silver Layer

In [30]:
# Connect to the Bronze layer to read raw data
conn_bronze = sqlite3.connect('bronze_layer.db')
df_bronze = pd.read_sql_query("SELECT * FROM customer_order", conn_bronze)
conn_bronze.close()

# Step 2.1: Data Cleaning
# Remove duplicates
df_silver = df_bronze.drop_duplicates()

# Convert order_date to datetime format
# Use .loc to avoid SettingWithCopyWarning and ensure safe modification
df_silver = df_silver.copy()  # Ensure it's a copy, not a view
df_silver['order_date'] = pd.to_datetime(df_silver['order_date'])

# Step 2.2: Filtering data - Keep only completed and pending orders
df_silver = df_silver[df_silver['status'].isin(['completed', 'pending'])]

# Step 2.3: Save the cleaned data to the Silver layer
conn_silver = sqlite3.connect('silver_layer.db')
df_silver.to_sql('customer_order', conn_silver, if_exists='replace', index=False)
conn_silver.close()


In [31]:
conn_silver = sqlite3.connect('silver_layer.db')
cursor = conn_silver.cursor()
cursor.execute('''SELECT * FROM customer_order;''')
# Fetch and print all the results
rows = cursor.fetchall()

# Check if there are results and display them
if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn_silver.close()

(1, 'C001', '2024-01-15 00:00:00', 250.5, 'completed')
(2, 'C002', '2024-02-20 00:00:00', 145.0, 'completed')
(3, 'C001', '2024-02-20 00:00:00', 300.0, 'pending')
(4, 'C003', '2024-03-05 00:00:00', 500.0, 'completed')
(6, 'C001', '2024-03-15 00:00:00', 275.0, 'completed')


# Gold Layer

In [32]:
# Connect to Silver layer database and read the data
conn_silver = sqlite3.connect('silver_layer.db')
df_silver = pd.read_sql_query('SELECT * FROM customer_order', conn_silver)

# Filter the data to include only completed orders
df_gold = df_silver[df_silver['status'] == 'completed']

# Perform aggregation: total amount spent and count of completed orders per customer
df_gold_summary = df_gold.groupby('customer_id').agg(
    total_spent=('amount', 'sum'),
    completed_orders=('order_id', 'count')
).reset_index()

# Create Gold layer database and save the aggregated data
conn_gold = sqlite3.connect('gold_layer.db')
df_gold_summary.to_sql('customer_summary', conn_gold, if_exists='replace', index=False)

# Close both connections
conn_silver.close()
conn_gold.close()

print("\nGold Layer Data:")
print(df_gold_summary)



Gold Layer Data:
  customer_id  total_spent  completed_orders
0        C001        525.5                 2
1        C002        145.0                 1
2        C003        500.0                 1
